In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI


In [2]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

In [3]:
openai = OpenAI()

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

PATH_TO_CHROME_DRIVER = 'B:\\Users\\ekfon\\chromeDriver\\chromedriver.exe'

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url

        options = Options()

        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")

        service = Service(PATH_TO_CHROME_DRIVER)
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)

        input("Please complete the verification in the browser and press Enter to continue...")
        page_source = driver.page_source
        driver.quit()

        soup = BeautifulSoup(page_source, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.get_text(separator="\n", strip=True)

In [5]:
system_prompt = "You are a state-of-the-art website analyzing assistant. \
You provide a short summary, and you ignore any navigation-related content. \
Your response is presented in nice markdown."

In [6]:
def user_prompt_for(website):
    user_prompt = f"The website you are looking at is titled {website.title}"
    user_prompt += "\nThe content of the website is as follows: \
Provide a short summary of this website. If the website contains news or announcements, \
summarize those, too.\n\n"
    user_prompt += website.text
    return user_prompt

In [7]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [8]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [9]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [10]:
display_summary("https://openai.com/")

Please complete the verification in the browser and press Enter to continue... 


# OpenAI Website Summary

OpenAI focuses on the development and exploration of artificial intelligence (AI) technologies that are safe and beneficial for humanity. The website emphasizes several key aspects:

## Mission and Purpose
- OpenAI aims to create safe AGI (Artificial General Intelligence) that will be advantageous to all people.

## Featured Products and Tools
- The site introduces multiple AI solutions including:
  - **ChatGPT**: An AI for conversational interactions and various tasks.
  - **Sora**: A new platform enhancing creativity by generating text, images, and video from user input.
  - **ChatGPT Pro and Enterprise**: Advanced versions tailored for business and educational needs with enhanced functionality.
  - **API Platform**: Allows integration of OpenAI models into different applications.

## Recent Announcements (News)
- **Partnerships and Collaborations**: OpenAI announced a partnership with Apple to enhance their AI capabilities (June 2024).
- **New Model Releases**: Introduction of the OpenAI o1 series that emphasizes improved reasoning in responses and tools for developers (Dec 2024).
- **Content Extensions**: Collaboration with Le Monde and Prisa Media to include more diverse news content in ChatGPT (March 2024).

## Research Contributions
- OpenAI is actively engaged in research to improve AI safety, governance, and functionality.

This summary captures the essence of OpenAI's objectives, product offerings, and recent developments in the field of AI.